In [ ]:
push!(LOAD_PATH, "/home/amir/Work/gdmrg/src/")
import Tmp
#using Plots
#using LaTeXStrings

In [ ]:
sz_half = Tmp.sz_half;
sp_half = Tmp.sp_half;
sm_half = Tmp.sm_half;

In [ ]:
### loading example files for illustration ###
#include("example-01-compareHeisVsRandom.jl")

In [ ]:
reload("Tmp")
include("test/runtests.jl")

In [ ]:
A = rand(4, 4)
matvecA!(v, v0) = A_mul_B!(v, A, v0)
@show Tmp.eigsfn(matvecA!, rand(4), false)
@show eigs(A)

In [ ]:
@time Tmp.rungutz(16, 400, compare=true)
Lx = 4
@time e,v = Tmp.groundstate_xxz_szblock(Lx)
@show e/Lx  

In [ ]:
dimM = binom(Lx, div(Lx, 2))

# szblock_states = Vector{Int64}(block_size)
#     index = 0
#     for state=0:2^Lx-1
#         if count_ones(state) == M
#             index += 1
#             szblock_states[index] = state
#         end
#     end
# v_dense = zeros(Float64, 16)

v_dense[[4, 6, 7, 10, 11, 13]] = v


tmp = transpose(reshape(transpose(v_dense), 8, 2))
display(tmp)
fact = svdfact(tmp)
display(fact[:U])
display(fact[:S])
display(fact[:Vt])
tmp = transpose(reshape(transpose(diagm(fact[:S])*fact[:Vt]), 4,4))
fact = svdfact(tmp)
display(fact[:U])
display(fact[:S])
display(fact[:Vt])

In [ ]:
Lx = 4
dim_szblock = binomial(Lx, div(Lx, 2))
@time e, v = Tmp.groundstate_xxz_szblock(Lx)

v = normalize(rand(dim_szblock))
display(v)

dim = div(dim_szblock, 2)
#transpose(reshape(transpose(v), dim, 2))
@show svd(transpose(v[1:dim,:]))
@show svd(transpose(v[dim:end,:]))

In [ ]:
Lx = 8
index = 0
for state=0:2^Lx-1
    if count_ones(state) == div(Lx, 2)
        index += 1
        println(rpad(index, 3, " "), lpad(state, 4, " "), " : ", bin(state, Lx))
    end
end

In [ ]:
Lx = 4
M = div(Lx, 2)

@time e, v = Tmp.groundstate_xxz_szblock(Lx, sz_total=div(Lx,2)-M)

#v = normalize(rand(dim_szblock))
#display(v)
totaldim = binomial(Lx, M)
@assert length(v) == totaldim

udim = binomial(Lx-1, M-1)
ddim = binomial(Lx-1, M)
uv = v[1:updim]
dv = v[updim+1:totaldim]

#transpose(reshape(transpose(v), dim, 2))
ufact = svdfact(transpose(uv[:,:]))
dfact = svdfact(transpose(dv[:,:]))

uv = diagm(ufact[:S]) * ufact[:Vt]
dv = diagm(dfact[:S]) * dfact[:Vt]

@show hcat(uv,dv)
display(uv)
display(dv)

n = 2
dims = [binomial(Lx-n, i) for i=0:n-1]